In [2]:
!pip install opencv-python mediapipe sklearn matplotlib

In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [4]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [6]:
def draw_landmarks(image, results):
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)# Draw right hand connections

In [7]:
def draw_styled_landmarks(image, results):
    # Draw face connections
   # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
    #                         mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
     #                        mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
      #                       )
        
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [8]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [9]:
results

mediapipe.python.solution_base.SolutionOutputs

In [10]:
draw_styled_landmarks(frame, results)

In [11]:
#len(results.left_hand_landmarks)

In [1]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

NameError: name 'plt' is not defined

In [13]:
#len(results.left_hand_landmarks.landmark)

In [14]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [15]:

pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [16]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [17]:
result_test = extract_keypoints(results)

In [18]:
result_test

array([ 0.56848586,  0.54653567, -1.10919511, ...,  0.        ,
        0.        ,  0.        ])

In [19]:
468*3+33*4+21*3+21*3

1662

In [20]:
np.save('0', result_test)

In [21]:
np.load('0.npy')

array([ 0.56848586,  0.54653567, -1.10919511, ...,  0.        ,
        0.        ,  0.        ])

In [22]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['hello', 'bathroom','alldone'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [23]:
for action in actions: 
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [23]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [24]:
cap.release()
cv2.destroyAllWindows()

In [25]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model

In [26]:
from tensorflow.compat.v1.losses import sparse_softmax_cross_entropy

In [27]:
label_map = {label:num for num, label in enumerate(actions)}

In [24]:
label_map

{'hello': 0, 'bathroom': 1, 'alldone': 2}

In [25]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [26]:
np.array(sequences).shape

(120, 30, 1662)

In [27]:
np.array(labels).shape

(120,)

In [28]:
X = np.array(sequences)

In [29]:
X.shape

(120, 30, 1662)

In [30]:
y = to_categorical(labels).astype(int)

In [31]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [33]:
y_test.shape

(6, 3)

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [35]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [36]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [37]:
res = [.7, 0.2, 0.1]

In [38]:
actions[np.argmax(res)]

'hello'

In [39]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [72]:
model.fit(X_train, y_train, epochs=700, callbacks=[tb_callback])

Epoch 1/700
4/4 [==============================] - 5s 138ms/step - loss: 1.3506 - categorical_accuracy: 0.3246
Epoch 2/700
4/4 [==============================] - 1s 181ms/step - loss: 1.5284 - categorical_accuracy: 0.3070
Epoch 3/700
4/4 [==============================] - 1s 200ms/step - loss: 1.0630 - categorical_accuracy: 0.3947
Epoch 4/700
4/4 [==============================] - 1s 120ms/step - loss: 0.9566 - categorical_accuracy: 0.5526
Epoch 5/700
4/4 [==============================] - 1s 159ms/step - loss: 1.4721 - categorical_accuracy: 0.4298
Epoch 6/700
4/4 [==============================] - 1s 117ms/step - loss: 1.0864 - categorical_accuracy: 0.3333
Epoch 7/700
4/4 [==============================] - 1s 118ms/step - loss: 1.1220 - categorical_accuracy: 0.4649
Epoch 8/700
4/4 [==============================] - 0s 120ms/step - loss: 1.0369 - categorical_accuracy: 0.4298
Epoch 9/700
4/4 [==============================] - 1s 125ms/step - loss: 1.0856 - categorical_accuracy: 0.3860
E

4/4 [==============================] - 1s 118ms/step - loss: 3953.6624 - categorical_accuracy: 0.3684
Epoch 145/700
4/4 [==============================] - 1s 128ms/step - loss: 10213.2646 - categorical_accuracy: 0.3246
Epoch 146/700
4/4 [==============================] - 0s 118ms/step - loss: 4581.7515 - categorical_accuracy: 0.2895
Epoch 147/700
4/4 [==============================] - 1s 123ms/step - loss: 4548.1167 - categorical_accuracy: 0.3246
Epoch 148/700
4/4 [==============================] - 1s 166ms/step - loss: 4586.7061 - categorical_accuracy: 0.2982
Epoch 149/700
4/4 [==============================] - 1s 119ms/step - loss: 2208.3044 - categorical_accuracy: 0.3333
Epoch 150/700
4/4 [==============================] - 1s 161ms/step - loss: 2512.9258 - categorical_accuracy: 0.2982
Epoch 151/700
4/4 [==============================] - 1s 164ms/step - loss: 3984.9871 - categorical_accuracy: 0.3596
Epoch 152/700
4/4 [==============================] - 1s 136ms/step - loss: 4611.2920 

4/4 [==============================] - 1s 137ms/step - loss: 112.8576 - categorical_accuracy: 0.3860
Epoch 287/700
4/4 [==============================] - 1s 121ms/step - loss: 166.5373 - categorical_accuracy: 0.3509
Epoch 288/700
4/4 [==============================] - 1s 127ms/step - loss: 97.2263 - categorical_accuracy: 0.3684
Epoch 289/700
4/4 [==============================] - 1s 131ms/step - loss: 95.7960 - categorical_accuracy: 0.3158
Epoch 290/700
4/4 [==============================] - 1s 126ms/step - loss: 124.0388 - categorical_accuracy: 0.3509
Epoch 291/700
4/4 [==============================] - 1s 134ms/step - loss: 42.8615 - categorical_accuracy: 0.4035
Epoch 292/700
4/4 [==============================] - 1s 122ms/step - loss: 124.2004 - categorical_accuracy: 0.3684
Epoch 293/700
4/4 [==============================] - 0s 123ms/step - loss: 103.8474 - categorical_accuracy: 0.3947
Epoch 294/700
4/4 [==============================] - 0s 124ms/step - loss: 150.4808 - categorical

4/4 [==============================] - 0s 117ms/step - loss: 3.0145 - categorical_accuracy: 0.6667
Epoch 430/700
4/4 [==============================] - 1s 140ms/step - loss: 3.6985 - categorical_accuracy: 0.6754
Epoch 431/700
4/4 [==============================] - 0s 122ms/step - loss: 3.5560 - categorical_accuracy: 0.6667
Epoch 432/700
4/4 [==============================] - 0s 116ms/step - loss: 3.1984 - categorical_accuracy: 0.7193
Epoch 433/700
4/4 [==============================] - 0s 115ms/step - loss: 4.6513 - categorical_accuracy: 0.6316
Epoch 434/700
4/4 [==============================] - 0s 111ms/step - loss: 7.5783 - categorical_accuracy: 0.5439
Epoch 435/700
4/4 [==============================] - 0s 124ms/step - loss: 5.6671 - categorical_accuracy: 0.6228
Epoch 436/700
4/4 [==============================] - 0s 114ms/step - loss: 6.1336 - categorical_accuracy: 0.4649
Epoch 437/700
4/4 [==============================] - 0s 114ms/step - loss: 3.2402 - categorical_accuracy: 0.53

4/4 [==============================] - 1s 120ms/step - loss: 1.2442 - categorical_accuracy: 0.7281
Epoch 574/700
4/4 [==============================] - 0s 126ms/step - loss: 1.0882 - categorical_accuracy: 0.7193
Epoch 575/700
4/4 [==============================] - 1s 131ms/step - loss: 0.7143 - categorical_accuracy: 0.7632
Epoch 576/700
4/4 [==============================] - 0s 121ms/step - loss: 0.6495 - categorical_accuracy: 0.7719
Epoch 577/700
4/4 [==============================] - 0s 117ms/step - loss: 1.2728 - categorical_accuracy: 0.7193
Epoch 578/700
4/4 [==============================] - 0s 119ms/step - loss: 2.0205 - categorical_accuracy: 0.7193
Epoch 579/700
4/4 [==============================] - 1s 125ms/step - loss: 4.5179 - categorical_accuracy: 0.6404
Epoch 580/700
4/4 [==============================] - 0s 115ms/step - loss: 3.4302 - categorical_accuracy: 0.6754
Epoch 581/700
4/4 [==============================] - 0s 122ms/step - loss: 6.7514 - categorical_accuracy: 0.64

In [51]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

In [40]:
res = model.predict(X_test)

1/1 [==============================] - 3s 3s/step


In [41]:
actions[np.argmax(res[3])]

'bathroom'

In [42]:
actions[np.argmax(y_test[3])]

'alldone'

In [43]:
model.save('action.h5')

In [56]:
#del model

In [44]:
model.load_weights('action.h5')

In [45]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [46]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 31ms/step


In [47]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [48]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[5., 0.],
        [1., 0.]],

       [[0., 6.],
        [0., 0.]],

       [[1., 0.],
        [5., 0.]]])

In [49]:
accuracy_score(ytrue, yhat)

0.0

In [50]:
colors = [(245,117,16), (117,245,16), (16,117,245),(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [124]:
multilabel_confusion_matrix(ytrue, yhat)

In [53]:
accuracy_score(ytrue, yhat)

1/1 [==============================] - 0s 160ms/step


array([[0.3436779 , 0.3557962 , 0.30052593]], dtype=float32)

In [54]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

(30, 1662)

In [1]:
from tkinter import*
from tkinter import messagebox
from tkinter import ttk

In [ ]:
def start():
    
    
    # 1. New detection variables
    sequence = []
    sentence = []
    predictions = []
    threshold = 0.5

    cap = cv2.VideoCapture(0)
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        
        while cap.isOpened():
            ret, frame = cap.read()

        # Make detections
            image, results = mediapipe_detection(frame, holistic)
            print(results)
        
        # Draw landmarks
            draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-30:]
        
            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))
            
            
        #3. Viz logic
                if np.unique(predictions[-10:])[0]==np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 

                        if len(sentence) > 0: 
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])

                if len(sentence) > 5: 
                    sentence = sentence[-5:]

            # Viz probabilities
                image = prob_viz(res, actions, image, colors)
            
                cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
                cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
                cv2.imshow('OpenCV Feed', image)

        # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                       break
cap.release()
cv2.destroyAllWindows()

In [ ]:
m=Toplevel()
m.title("Language Detection")
m.geometry("700x400")
f1=Frame(m,bd=5,bg='orange')
f1.place(x=1,y=102,width=1400,height=900)
l1=Label(m,text='WELCOME TO HAND GESTURE DETECTOR',bg='brown',fg='pink',font=('Ariel',38),pady=20)
l1.pack(side=TOP,fill="x")
img=PhotoImage(file=r"C:\Users\DANIYAL1\Desktop\SquarePic_20230116_18562120.jpg")

img2=img.subsample(3,3)
l2=Label(f1,image=img2).grid(row=0,column=0,padx=37,pady=20)
img=PhotoImage(file=r"C:\Users\DANIYAL1\Desktop\SquarePic_20230116_18574323.jpg")

img4=img.subsample(3,3)
l3=Label(f1,image=img4).grid(row=0,column=1,padx=37,pady=20)
img=PhotoImage(file=r"C:\Users\DANIYAL1\Desktop\SquarePic_20230116_18510189.jpg")

img6=img.subsample(3,3)
l4=Label(f1,image=img6).grid(row=0,column=2,padx=37,pady=20)


b1=Button(f1,text="START",bg='brown',fg='white',command=start,width=13,font=('Ariel',20)).grid(row=2,column=1,pady=50,padx=32)
m.mainloop()